In [ ]:
!pip install -q \
  torch>=2.1.0 \
  transformers>=4.39.0 \
  appdirs \
  jsonpickle \
  filelock \
  h5py \
  nltk \
  dotmap \
  pytest


In [ ]:
!pip install radgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.0/588.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for radgraph: filename=radgraph-0.1.18-py3-none-any.whl size=812635 sha256=49a6daa0f705ad8e1280c4c577ea6c8c5e8dfaf11233e533c30bae22c91df73a
  Stored in directory: /root/.cache/pip/wheels/fb/3c/fb/214f5d5cdab2a0f9f0904fd81d7fd1134404100b4444554df8
Successfully built radgraph


In [ ]:
import json
from radgraph import get_radgraph_processed_annotations, RadGraph

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
import pandas as pd
import ast
import re


In [ ]:

df = pd.read_csv("/content/output.csv")

clean = (
    df.iloc[0:50, 0]
      .astype(str)
      .str.replace(r'^FINDINGS:\s*', '', regex=True)
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
      .reset_index(drop=True)
)

concepts = df.iloc[0:50, 1].reset_index(drop=True)

eval_df = pd.DataFrame({"report": clean, "concepts": concepts})
model_type = "modern-radgraph-xl"
radgraph = RadGraph(model_type=model_type)
reports = eval_df["report"].astype(str).tolist()

anno = []= []

for report in reports:
    annotations = radgraph([report])  # one report batch
    processed = get_radgraph_processed_annotations(annotations)  # dict with "processed_annotations"
    anno.append(processed)

print("num examples:", len(anno))


Using device: cpu


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


num examples: 50


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans
from tqdm.auto import tqdm

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:

nlp_tok = spacy.blank("en")

def map_label(rg_label: str):
    # "Observation::definitely absent" / "Anatomy::definitely present"
    typ, status = rg_label.split("::", 1)
    status = status.lower()

    if typ == "Anatomy":
        return "ANATOMY"

    if typ == "Observation":
        if "absent" in status:
            return "OBS_ABSENT"
        if "uncertain" in status or "possible" in status:
            return "OBS_UNCERTAIN"
        return "OBS_PRESENT"

    return None

docbin = DocBin(store_user_data=False)

for processed in tqdm(anno, desc="Building spaCy dataset"):
    rg0 = processed["radgraph_annotations"]["0"]
    text = rg0["text"]
    ents = rg0["entities"]

    doc = nlp_tok.make_doc(text)
    spans = []

    for ent in ents.values():
        label = map_label(ent["label"])
        if label is None:
            continue

        start_ix = int(ent["start_ix"])
        end_ix   = int(ent["end_ix"])

        start_char = doc[start_ix].idx
        end_char   = doc[end_ix].idx + len(doc[end_ix])

        sp = doc.char_span(start_char, end_char, label=label, alignment_mode="contract")
        if sp is not None:
            spans.append(sp)

    doc.ents = filter_spans(spans)
    docbin.add(doc)

train_path = "/content/radgraph_train.spacy"
docbin.to_disk(train_path)
print("Saved:", train_path)


Building spaCy dataset:   0%|          | 0/50 [00:00<?, ?it/s]

Saved: /content/radgraph_train.spacy


In [ ]:
!python -m spacy init config /content/config.cfg --lang en --pipeline ner --optimize efficiency --force
!python -m spacy train /content/config.cfg \
  --output /content/spacy_student \
  --paths.train /content/radgraph_train.spacy \
  --paths.dev /content/radgraph_train.spacy


⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
/content/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
✔ Created output directory: /content/spacy_student
ℹ Saving to output directory: /content/spacy_student
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------

In [ ]:
import spacy
nlp = spacy.load("/content/spacy_student/model-best")

text = "The lungs are clear. No pleural effusion is seen."
doc = nlp(text)
[(ent.text, ent.label_) for ent in doc.ents]


[('lungs', 'ANATOMY'),
 ('clear', 'OBS_PRESENT'),
 ('pleural', 'ANATOMY'),
 ('effusion', 'OBS_ABSENT')]